# FEA Algebra Scratchbook

The purpose of this notebook is to serve as a quick area to do math (symbolic and numerical) with the shape functions and other expressions common in FEA. 
This is a substitute to trying to do everything by hand and calculator to save time

The second purpose is to output testing files for matrix and vector comparison.
This, again, saves a lot of time since these things don't have to be done fully by hand. 

## Mailroom

Importing necessary packages

In [1]:
import numpy as np
import scipy as sp
import sympy as smp

from sympy import Matrix
from scipy.integrate import fixed_quad
from numpy.linalg import inv

## Setup of Shape Functions, Derivatives, and Jacobian and Conversion Equations

The following cells contain the equations for the shape functions and their derivatives, as well as the Jacobian formula and isoparametric-to-physical formula for the L2 and L3 line elements.

In [2]:
# L2 Element
## Shape Functions ##
N0_L2 = lambda x: (1 - x)/2
N1_L2 = lambda x: (1 + x)/2

## Shape Function Derivatives ##
N0_D_L2 = lambda x: -1/2
N1_D_L2 = lambda x: 1/2

## Jacobian ##
J_L2 = lambda x, x1, x2: N0_D_L2(x)*x1 + N1_D_L2(x)*x2

## Isoparametric-to-Physical Converter ##
x_L2 = lambda x, x1, x2: N0_L2(x)*x1 + N1_L2(x)*x2

# L3 Element
## Shape Functions ##
N0_L3 = lambda x: 0.5*(x**2 - x)
N1_L3 = lambda x: 1 - x**2
N2_L3 = lambda x: 0.5*(x**2 + x)

## Shape Function Derivatives ##
N0_D_L3 = lambda x: x - 0.5
N1_D_L3 = lambda x: -2*x
N2_D_L3 = lambda x: x + 0.5  

## Jacobian ##
J_L3 = lambda x, x1, x2, x3: N0_D_L3(x)*x1 + N1_D_L3(x)*x2 + N2_D_L3(x)*x3

## Isoparametric-to-Physical Converter ##
x_L3 = lambda x, x1, x2, x3: N0_L3(x)*x1 + N1_L3(x)*x2 + N2_L3(x)*x3



N_L2 = [N0_L2, N1_L2]
N_D_L2 = [N0_D_L2, N1_D_L2]


N_L3 = [N0_L3, N1_L3, N2_L3]
N_D_L3 = [N0_D_L3, N1_D_L3, N2_D_L3]


## Element Matrix and Vector Composition Functions

The below functions assemble the matrix and vector arrays for the L2 and L3 line elements given the coordinates of the nodes and, depending on the function, either the constants or driving function used for the problem. 

In [3]:
# ODE Integrands
def k_ij_L2(x1, x2, A, B):
    return_array = np.zeros((2, 2))
    # Assemble the integrand here
    for i in [0, 1]:
        for j in [0, 1]:
            integrand = lambda x: -1*N_D_L2[i](x)*N_D_L2[j](x)*(1/J_L2(x, x1, x2)) + A*N_L2[i](x)*N_D_L2[j](x) + B*N_L2[i](x)*N_L2[j](x)*J_L2(x, x1, x2)
            return_array[i, j] += fixed_quad(integrand, -1, 1, n=9)[0]

    return return_array

def f_i_L2(x1, x2, driving_func):
    return_array = np.zeros(2)
    for i in [0, 1]:
        integrand = lambda x: driving_func(x_L2(x, x1, x2))*N_L2[i](x)*J_L2(x, x1, x2)
        return_array[i] += fixed_quad(integrand, -1, 1, n=9)[0]

    return return_array

def k_ij_L3(x1, x2, x3, A, B):
    return_array = np.zeros((3, 3))
    # Assemble the integrand here
    for i in [0, 1, 2]:
        for j in [0, 1, 2]:
            integrand = lambda x: -1*N_D_L3[i](x)*N_D_L3[j](x)*(1/J_L3(x, x1, x2, x3)) + A*N_L3[i](x)*N_D_L3[j](x) + B*N_L3[i](x)*N_L3[j](x)*J_L3(x, x1, x2, x3)
            return_array[i, j] += fixed_quad(integrand, -1, 1, n=10)[0]

    return return_array

def f_i_L3(x1, x2, x3, driving_func):
    return_array = np.zeros(3)
    for i in [0, 1, 2]:
        integrand = lambda x: driving_func(x_L3(x, x1, x2, x3))*N_L3[i](x)*J_L3(x, x1, x2, x3)
        return_array[i] += fixed_quad(integrand, -1, 1, n=10)[0]

    return return_array
            

These are auxiliary functions; they print out the arrays in either human-readable or binary format.
The binary format prints in such a way that it is readable by GSL.

In [4]:
# Helper Functions
def output_array(filename, array):
    size = array.shape[0]
    # Determine if the array is a vector or matrix
    if len(array.shape) == 2:
        f = open(filename, "w")
        for i in range(size):
            for j in range(size):
                f.write(f"{array[i, j]}")
                f.write("\n")
        f.close()
    elif len(array.shape) == 1:
        f = open(filename, "w")
        for i in range(size):
            f.write(f"{array[i]}")
            f.write("\n")

        f.close()

def output_array_binary(filename, array):
    with open(filename, "wb") as f:
        array.tofile(f)
        

## Output Test Matrices and Vectors

These cells both serve as the scratchwork and the test file outputs for the integration tests that the program uses.

In [5]:
linear_mesh = [0, 2.3, 3, 5.5, 10]
A = 4
B = 4
g = lambda x: x**2 + x + 3

K_list = []
F_list = []

for i in range(1, 5):
    x1 = linear_mesh[i - 1]
    x2 = linear_mesh[i]

    print(f"Element {i}")
    K_m = k_ij_L2(x1, x2, A, B)
    F_v = f_i_L2(x1, x2, g)

    """ Deprecated
    output_array(f"./test/integration/test_reference_array/L2/linear_matrix_{i}.txt", K_m)
    output_array(f"./test/integration/test_reference_array/L2/linear_vector_{i}.txt", F_v)
    """

    display(Matrix(K_m))
    display(Matrix(F_v))
    
    K_list.append(K_m)
    F_list.append(F_v)

Element 1


Matrix([
[  0.631884057971014, 3.96811594202898],
[-0.0318840579710144, 4.63188405797101]])

Matrix([
[5.34558333333333],
[8.25508333333333]])

Element 2


Matrix([
[ -2.49523809523809, 3.89523809523809],
[-0.104761904761905, 1.50476190476191]])

...

Matrix([
[ 0.933333333333333, 4.06666666666667],
[0.0666666666666668, 4.93333333333333]])

Matrix([
[        27.34375],
[37.2395833333333]])

Element 4


Matrix([
[3.77777777777778, 5.22222222222222],
[1.22222222222222, 7.77777777777778]])

Matrix([
[135.28125],
[190.96875]])

In [6]:
size = len(linear_mesh)
K_global = np.zeros((size, size), dtype = np.float64)
F_global = np.zeros(size, dtype = np.float64)

for i in range(4):
    index = i + 1
    K_global[index - 1:index + 1, index - 1:index + 1] += K_list[i]
    F_global[index - 1:index + 1] += F_list[i]

display(Matrix(K_global))
display(Matrix(F_global))

output_array_binary("./test/integration/test_reference_array/linear_matrix_global.output", K_global)
output_array_binary("./test/integration/test_reference_array/linear_vector_global.output", F_global)

Matrix([
[  0.631884057971014,   3.96811594202898,                0.0,              0.0,              0.0],
[-0.0318840579710144,   2.13664596273292,   3.89523809523809,              0.0,              0.0],
[                0.0, -0.104761904761905,   2.43809523809524, 4.06666666666667,              0.0],
[                0.0,                0.0, 0.0666666666666668, 8.71111111111111, 5.22222222222222],
[                0.0,                0.0,                0.0, 1.22222222222222, 7.77777777777778]])

Matrix([
[5.34558333333333],
[         12.4475],
[32.0506666666667],
[172.520833333333],
[       190.96875]])

In [7]:
# Output the LaTeX for testing purposes
print(smp.latex(Matrix(K_global).evalf(5)))
print(smp.latex(Matrix(F_global).evalf(5)))

\left[\begin{matrix}0.63188 & 3.9681 & 0 & 0 & 0\\-0.031884 & 2.1366 & 3.8952 & 0 & 0\\0 & -0.10476 & 2.4381 & 4.0667 & 0\\0 & 0 & 0.066667 & 8.7111 & 5.2222\\0 & 0 & 0 & 1.2222 & 7.7778\end{matrix}\right]
\left[\begin{matrix}5.3456\\12.447\\32.051\\172.52\\190.97\end{matrix}\right]


In [8]:
# Get solution vector
# Set the boundary conditions
d1 = 0
d2 = 5
K_global[0, :] = 0
K_global[-1, :] = 0
K_global[0, 0] = 1
K_global[-1, -1] = 1

F_global[0] = d1
F_global[-1] = d2

solution_vector = inv(K_global)@F_global
display(Matrix(solution_vector))
output_array_binary("./test/integration/test_reference_array/linear_solution_vector.output", solution_vector)

Matrix([
[              0.0],
[ 30.8693165657188],
[-13.7371065142032],
[ 16.9123694376087],
[              5.0]])

In [9]:
# Output the LaTeX for testing purposes
print(smp.latex(Matrix(solution_vector).evalf(5)))

\left[\begin{matrix}0\\30.869\\-13.737\\16.912\\5.0\end{matrix}\right]


In [10]:
quadratic_mesh = [0, 1.25, 2.5, 3, 6, 6.8, 7, 8.5, 10]
A = 4
B = 4
g = lambda x: x**2 + x + 3

K_list = []
F_list = []

for i in range(2, 9, 2):
    x1 = quadratic_mesh[i - 2]
    x2 = quadratic_mesh[i - 1]
    x3 = quadratic_mesh[i]

    index = int((i - 2)/2 + 1)

    print(f"Element {index}")
    K_m = k_ij_L3(x1, x2, x3, A, B)
    F_v = f_i_L3(x1, x2, x3, g)

    """ Deprecated
    output_array(f"./test/integration/test_reference_array/L3/quad_matrix_{index}.txt", K_m)
    output_array(f"./test/integration/test_reference_array/L3/quad_vector_{index}.txt", F_v)
    """
  
    display(Matrix(K_m))
    display(Matrix(F_v))

    K_list.append(K_m)
    F_list.append(F_v)


Element 1


Matrix([
[              -1.6,   4.39999999999999, -1.13333333333333],
[-0.933333333333339,   3.20000000000001,               4.4],
[ 0.199999999999999, -0.933333333333329,  2.39999999999999]])

Matrix([
[0.98958333333333],
[10.2083333333333],
[4.63541666666666]])

Element 2


Matrix([
[ -7.63777874642091,  8.35518631526884, -1.71740756884793],
[  3.02185298193551, 0.363393743297469,  5.94808660810037],
[-0.384074235514599, 0.614753274767037,  5.43598762741422]])

...

Matrix([
[-1.19540087819104, -0.702995609044803,   3.3650631539025],
[-6.03632894237813,   21.9149780452239, -13.2119824361791],
[ 4.69839648723583,  -18.5453157695125,  13.7135859489433]])

Matrix([
[ 17.6165904761904],
[ 35.6670476190476],
[-1.45030476190476]])

Element 4


Matrix([
[ -1.17777777777778,   4.35555555555555, -1.17777777777778],
[-0.977777777777782,   4.62222222222224,  4.35555555555556],
[ 0.155555555555555, -0.977777777777773,  2.82222222222221]])

Matrix([
[           29.05],
[           168.4],
[56.0499999999998]])

In [11]:
size = len(quadratic_mesh)
K_global = np.zeros((size, size), dtype = np.float64)
F_global = np.zeros(size, dtype = np.float64)

for i in range(2, size, 2):
    index = 2*i + 1
    K_global[i - 2:i + 1, i - 2:i + 1] += K_list[(i - 1)//2]
    F_global[i - 2:i + 1] += F_list[(i - 1)//2]

display(Matrix(K_global))
display(Matrix(F_global))

output_array_binary("./test/integration/test_reference_array/quad_matrix_global.output", K_global)
output_array_binary("./test/integration/test_reference_array/quad_vector_global.output", F_global)

Matrix([
[              -1.6,   4.39999999999999,  -1.13333333333333,               0.0,               0.0,                0.0,                0.0,                0.0,               0.0],
[-0.933333333333339,   3.20000000000001,                4.4,               0.0,               0.0,                0.0,                0.0,                0.0,               0.0],
[ 0.199999999999999, -0.933333333333329,  -5.23777874642091,  8.35518631526884, -1.71740756884793,                0.0,                0.0,                0.0,               0.0],
[               0.0,                0.0,   3.02185298193551, 0.363393743297469,  5.94808660810037,                0.0,                0.0,                0.0,               0.0],
[               0.0,                0.0, -0.384074235514599, 0.614753274767037,  4.24058674922318, -0.702995609044803,    3.3650631539025,                0.0,               0.0],
[               0.0,                0.0,                0.0,               0.0, -6.0363289423781

Matrix([
[ 0.98958333333333],
[ 10.2083333333333],
[-2.04345238095239],
[ 50.2416666666667],
[ 66.2204595238093],
[ 35.6670476190476],
[ 27.5996952380952],
[            168.4],
[ 56.0499999999998]])

In [12]:
# Output the LaTeX for testing purposes
print(smp.latex(Matrix(K_global).evalf(5)))
print(smp.latex(Matrix(F_global).evalf(5)))

\left[\begin{matrix}-1.6 & 4.4 & -1.1333 & 0 & 0 & 0 & 0 & 0 & 0\\-0.93333 & 3.2 & 4.4 & 0 & 0 & 0 & 0 & 0 & 0\\0.2 & -0.93333 & -5.2378 & 8.3552 & -1.7174 & 0 & 0 & 0 & 0\\0 & 0 & 3.0219 & 0.36339 & 5.9481 & 0 & 0 & 0 & 0\\0 & 0 & -0.38407 & 0.61475 & 4.2406 & -0.703 & 3.3651 & 0 & 0\\0 & 0 & 0 & 0 & -6.0363 & 21.915 & -13.212 & 0 & 0\\0 & 0 & 0 & 0 & 4.6984 & -18.545 & 12.536 & 4.3556 & -1.1778\\0 & 0 & 0 & 0 & 0 & 0 & -0.97778 & 4.6222 & 4.3556\\0 & 0 & 0 & 0 & 0 & 0 & 0.15556 & -0.97778 & 2.8222\end{matrix}\right]
\left[\begin{matrix}0.98958\\10.208\\-2.0435\\50.242\\66.22\\35.667\\27.6\\168.4\\56.05\end{matrix}\right]


In [13]:
# Get solution vector
# Set the boundary conditions
d1 = 0
d2 = 5
K_global[0, :] = 0
K_global[-1, :] = 0
K_global[0, 0] = 1
K_global[-1, -1] = 1

F_global[0] = d1
F_global[-1] = d2

solution_vector = inv(K_global)@F_global
display(Matrix(solution_vector))
output_array_binary("./test/integration/test_reference_array/quad_solution_vector.output", solution_vector)

Matrix([
[              0.0],
[ 69.2757324291173],
[-48.0622750999643],
[-15.6835465931684],
[ 33.8223216902721],
[-5.11199448515188],
[-26.6318060552635],
[ 26.0875025652327],
[              5.0]])

In [14]:
# Output the LaTeX for testing purposes
print(smp.latex(Matrix(solution_vector).evalf(5)))

\left[\begin{matrix}0\\69.276\\-48.062\\-15.684\\33.822\\-5.112\\-26.632\\26.087\\5.0\end{matrix}\right]


In [52]:
print(k_ij_L2(1, 3, 4, 4))
print(f_i_L2(1, 3, g))
print(k_ij_L3(0, 1.3, 3, 4, 4))
print(f_i_L3(0, 1.3, 3, g))

[[ 0.16666667  3.83333333]
 [-0.16666667  4.16666667]]
[ 7.66666667 11.        ]
[[-1.64986424  4.30603493 -1.18950403]
 [-1.0272984   4.54229129  4.48500711]
 [ 0.14382931 -0.84832623  3.23783025]]
[ 0.44230476 13.77828571  8.27940952]
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Random stuff below.

In [15]:
for func, input_v in zip(N_L3, [-0.43, -0.99, 0.48]):
    foo = func(input_v)
    print(foo)

0.30745
0.01990000000000003
0.35519999999999996


In [16]:
for func, input_v in zip(N_D_L3, [0.13, -0.13, 0.57]):
    foo = func(input_v)
    print(foo)

-0.37
0.26
1.0699999999999998


In [17]:
foo = J_L3(-0.07, 6, 6.8, 7)
bar = x_L3(0.1, 6, 6.8, 7)

print(foo)
print(bar)

0.5419999999999994
6.8469999999999995


In [32]:
driving_func = lambda x: x**2 + x + 3
input = 0.2
ans = driving_func(x_L2(input, 1, 3))*N_L2[1](input)*J_L2(input, 1, 3)
print(ans)


6.023999999999999


In [41]:
A = 4
B = 4
i = 1
j = 1

integrand = lambda x: -1*N_D_L2[i](x)*N_D_L2[j](x)*(1/J_L2(x, 1, 3)) + A*N_L2[i](x)*N_D_L2[j](x) + B*N_L2[i](x)*N_L2[j](x)*J_L2(x, 1, 3)

print(integrand(-0.75))


0.0625


In [45]:
driving_func = lambda x: x**2 + x + 3
input = 0.2
for input, i in zip([-0.3, 0.35, 0.55], [0, 1, 2]):
    ans = driving_func(x_L3(input, 0, 1.3, 3))*N_L3[i](input)*J_L3(input, 0, 1.3, 3)
    print(ans)


1.2436251984000002
11.901573224775
7.303574499787502


In [49]:
input_array = np.array([
    [-0.1, -0.5, -0.75],
    [0.3, 0.65, -0.1],
    [0, 0.9, -0.53]
])

for i in [0, 1, 2]:
    for j in [0, 1, 2]:
        inpt = input_array[i][j]
        integrand = lambda x: -1*N_D_L3[i](x)*N_D_L3[j](x)*(1/J_L3(x, 0, 1.3, 3)) + A*N_L3[i](x)*N_D_L3[j](x) + B*N_L3[i](x)*N_L3[j](x)*J_L3(x, 0, 1.3, 3)
        print(round(integrand(inpt), 6))




-0.360909
3.731731
-1.211979
-1.421238
-1.615343
1.269033
0.166667
-3.592533
0.094169


In [50]:
input_array = np.array([
    [-0.75, 0.1],
    [0.75, -0.2],
])

for i in [0, 1]:
    for j in [0, 1]:
        inpt = input_array[i][j]
        integrand = lambda x: -1*N_D_L2[i](x)*N_D_L2[j](x)*(1/J_L2(x, 1, 3)) + A*N_L2[i](x)*N_D_L2[j](x) + B*N_L2[i](x)*N_L2[j](x)*J_L2(x, 1, 3)
        print(round(integrand(inpt), 6))




1.0625
2.14
-1.0625
1.19
